<a href="https://colab.research.google.com/github/JuanZapa7a/semiotics/blob/main/larq1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pasos para Implementar y Comparar VGG8 Normal vs. Hardware-Aware en Larq

**Larq** es una librería diseñada para construir y entrenar redes neuronales binarizadas (BNNs) utilizando TensorFlow y Keras. Si estás interesado en realizar un entrenamiento consciente del hardware (considerando ruido, cuantización, etc.) para **VGG8** usando **Larq**, puedes lograrlo aprovechando las funcionalidades específicas de **Larq** para la binarización y entrenamiento de modelos compactos.

In [1]:
pip install tensorflow==2.14.0 larq==0.13.0

In [3]:
import tensorflow as tf
import larq as lq

print(tf.__version__)
print(lq.__version__)

2.14.0
0.13.0


#### **2. Definición del Modelo VGG8 Normal**
Primero, definimos la arquitectura estándar de **VGG8** en Larq sin binarización:

In [4]:
import tensorflow as tf
import larq as lq
from tensorflow.keras import layers, models

# Arquitectura de VGG8 estándar en Larq (sin binarización)
def build_vgg8(input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential([
        layers.Input(input_shape),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model_vgg8 = build_vgg8()

#### **3. Definición del Modelo Hardware-Aware VGG8**
En este modelo, implementamos binarización de pesos y activaciones utilizando **Larq** para simular hardware de baja precisión.

In [5]:
# Arquitectura de VGG8 con binarización (Hardware-Aware)
def build_hw_aware_vgg8(input_shape=(32, 32, 3), num_classes=10):
    kwargs = dict(input_quantizer="ste_sign", kernel_quantizer="ste_sign", kernel_constraint="weight_clip")
    model = models.Sequential([
        lq.layers.QuantConv2D(64, (3, 3), padding='same', activation='relu', **kwargs),
        layers.MaxPooling2D((2, 2)),
        lq.layers.QuantConv2D(128, (3, 3), padding='same', activation='relu', **kwargs),
        layers.MaxPooling2D((2, 2)),
        lq.layers.QuantConv2D(256, (3, 3), padding='same', activation='relu', **kwargs),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        lq.layers.QuantDense(256, activation='relu', **kwargs),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model_hw_aware_vgg8 = build_hw_aware_vgg8()

#### **4. Compilación de los Modelos**
Compilamos ambos modelos utilizando el optimizador Adam y la pérdida de entropía cruzada.

In [6]:
model_vgg8.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_hw_aware_vgg8.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### **5. Preparación del Dataset CIFAR-10**
Utilizaremos **CIFAR-10** para entrenar ambos modelos. Realizamos una normalización básica en las imágenes.

In [7]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Carga de datos y preprocesamiento
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 [==============================] - 18s 0us/step


#### **6. Entrenamiento de los Modelos**
Entrenamos ambos modelos con los mismos datos y evaluamos su rendimiento.

In [ ]:
epochs = 10
batch_size = 64

# Entrenamiento del modelo normal
history_vgg8 = model_vgg8.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

# Entrenamiento del modelo Hardware-Aware
history_hw_vgg8 = model_hw_aware_vgg8.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Epoch 1/10
270/782 [=========>....................] - ETA: 3:06 - loss: 1.6684 - accuracy: 0.3914